In [1]:
!pip install deepface
!pip install tensorflow==2.10
!pip install flask-ngrok
!pip install pyngrok

In [2]:
from deepface import DeepFace
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os

In [4]:
# # Load and compare two images
# result = DeepFace.verify("/content/user_image.jpg", "/content/reference_image3.jpg", model_name="Facenet")

# print("Is same person: ", result["verified"])

Is same person:  True


In [21]:
!ngrok config add-authtoken 2nw6FAnRwjT0D5E1DQWfqnLEMtC_3gpaWgbyGRfqfnpCxPNDf # My AuthToken on ngrok API gateway

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://78ae-34-138-81-110.ngrok-free.app" -> "http://localhost:5000"


In [23]:
from flask import Flask, request, jsonify
import os
from deepface import DeepFace

app = Flask(__name__)

@app.route("/verify", methods=["POST"])
def verify():
    if "user_image" not in request.files or "reference_image" not in request.files:
        return jsonify({"error": "Please provide both 'user_image' and 'reference_image' files."}), 400

    user_image = request.files["user_image"]
    reference_image = request.files["reference_image"]

    # Save images temporarily in Colab's file system
    user_image_path = "/content/user_image.jpg"
    reference_image_path = "/content/reference_image.jpg"
    user_image.save(user_image_path)
    reference_image.save(reference_image_path)

    # Perform face verification
    try:
        result = DeepFace.verify(user_image_path, reference_image_path, model_name="Facenet")
        is_same_person = result["verified"]
        return jsonify({"is_same_person": is_same_person})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally:
        # Clean up the saved images
        os.remove(user_image_path)
        os.remove(reference_image_path)

In [24]:
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)  # Change to appropriate port if needed

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Oct/2024 16:57:34] "POST /verify HTTP/1.1" 200 -
